prereqs:
- download https://github.com/matt-raporte/anli and save it to your drive under ColabNotebooks/anli
- supress disconects step #7 https://towardsdatascience.com/10-tips-for-a-better-google-colab-experience-33f8fe721b82


In [1]:
# Mount into drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
%%bash
pip install torch==1.7
pip install transformers==3.0.2
pip install flint
pip install tqdm

  Using cached torch-1.7.0-cp37-cp37m-manylinux1_x86_64.whl (776.7 MB)
  Created wheel for flint: filename=flint-0.2-py3-none-any.whl size=6466 sha256=a39a0664e9c761ca5aeb712ef41ed8eb9a5d7c185de2b5427ffb9daebcd5000b
  Stored in directory: /root/.cache/pip/wheels/f4/c1/1f/33c8728944adbac46dcbfb47bc6587fa0b09dd22566061ae57
Successfully built flint


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.7.0 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.7.0 which is incompatible.


In [3]:
%%bash
cd drive/MyDrive/ColabNotebooks/anli
source setup.sh
bash script/download_data.sh
python src/dataset_tools/build_data.py

PYTHONPATH=/env/python:/content/drive/MyDrive/ColabNotebooks/anli/src:/content/drive/MyDrive/ColabNotebooks/anli/utest
The path of project root: /content/drive/MyDrive/ColabNotebooks/anli
SNLI checked.
MNLI checked.
FEVER NLI checked.
ANLI checked.
Data download completed and checked.
Load Jsonl: /content/drive/MyDrive/ColabNotebooks/anli/data/snli_1.0/snli_1.0_train.jsonl
Load Jsonl: /content/drive/MyDrive/ColabNotebooks/anli/data/snli_1.0/snli_1.0_dev.jsonl
Load Jsonl: /content/drive/MyDrive/ColabNotebooks/anli/data/snli_1.0/snli_1.0_test.jsonl
SNLI examples without gold label have been filtered.
SNLI Train size: 549367
SNLI Dev size: 9842
SNLI Test size: 9824
Save to Jsonl: /content/drive/MyDrive/ColabNotebooks/anli/data/build/snli/train.jsonl
Save to Jsonl: /content/drive/MyDrive/ColabNotebooks/anli/data/build/snli/dev.jsonl
Save to Jsonl: /content/drive/MyDrive/ColabNotebooks/anli/data/build/snli/test.jsonl
Load Jsonl: /content/drive/MyDrive/ColabNotebooks/anli/data/multinli_1.0/m

550152it [00:09, 61088.39it/s]
10000it [00:00, 12658.27it/s]
10000it [00:01, 7257.42it/s]
392702it [00:09, 39724.31it/s]
10000it [00:01, 8680.49it/s]
10000it [00:00, 12592.06it/s]
208346it [00:04, 48752.12it/s]
19998it [00:00, 24285.96it/s]
19998it [00:00, 27896.04it/s]
16946it [00:00, 26249.00it/s]
1000it [00:00, 2653.31it/s]
1000it [00:00, 2843.94it/s]
45460it [00:01, 38920.85it/s]
1000it [00:00, 2724.31it/s]
1000it [00:00, 2563.41it/s]
100459it [00:02, 36878.88it/s]
1200it [00:00, 3316.38it/s]
1200it [00:00, 3114.44it/s]


In [4]:
#flint.data_utils.fields -> import wasn't working for some reason

# Copyright (c) Facebook, Inc. and its affiliates.
#
# This source code is licensed under Creative Commons-Non Commercial 4.0 found in the
# LICENSE file in the root directory of this source tree.

import torch


class FlintField(object):
    @classmethod
    def batching(cls, batched_data):
        raise NotImplemented()


class RawFlintField(FlintField):
    @classmethod
    def batching(cls, batched_data):
        return batched_data


class LabelFlintField(FlintField):
    def batching(self, batched_data):
        return torch.tensor(batched_data)


class ArrayIndexFlintField(FlintField):
    def __init__(self, pad_idx, eos_idx=None, left_pad=False, move_eos_to_beginning=False) -> None:
        super().__init__()
        self.pad_idx = pad_idx
        self.eos_idx = eos_idx
        self.left_pad = left_pad
        self.move_eos_to_beginning = move_eos_to_beginning

    def collate_tokens(self, values, pad_idx, eos_idx=None, left_pad=False, move_eos_to_beginning=False):
        """
        Convert a list of 1d tensors into a padded 2d tensor.
        """
        if not torch.is_tensor(values[0]):
            values = [torch.tensor(v) for v in values]

        size = max(v.size(0) for v in values)
        res = values[0].new(len(values), size).fill_(pad_idx)

        def copy_tensor(src, dst):
            assert dst.numel() == src.numel()
            if move_eos_to_beginning:
                assert src[-1] == eos_idx
                dst[0] = eos_idx
                dst[1:] = src[:-1]
            else:
                dst.copy_(src)

        for i, v in enumerate(values):
            copy_tensor(v, res[i][size - len(v):] if left_pad else res[i][:len(v)])
        return res

    def batching(self, batched_data):
        return self.collate_tokens(batched_data,
                                   self.pad_idx,
                                   self.eos_idx,
                                   self.left_pad,
                                   self.move_eos_to_beginning)


# Copyright (c) Facebook, Inc. and its affiliates.
#
# This source code is licensed under Creative Commons-Non Commercial 4.0 found in the
# LICENSE file in the root directory of this source tree.

import torch
from typing import Dict, Type


class BaseBatchBuilder(object):
    def __init__(self, batching_schema: Dict[str, FlintField]) -> None:
        super().__init__()
        self.batching_schema: Dict[str, FlintField] = batching_schema

    def __call__(self, batch):
        field_names = batch[0].keys()
        batched_data = dict()

        for field_name in field_names:
            if field_name not in self.batching_schema:
                # default is RawFlintField
                batched_data[field_name] = RawFlintField.batching([item[field_name] for item in batch])

            else:
                batched_data[field_name] = self.batching_schema[field_name].batching([item[field_name] for item in batch])

        return batched_data


def has_tensor(obj) -> bool:
    """
    Given a possibly complex data structure,
    check if it has any torch.Tensors in it.
    """
    if isinstance(obj, torch.Tensor):
        return True
    elif isinstance(obj, dict):
        return any(has_tensor(value) for value in obj.values())
    elif isinstance(obj, (list, tuple)):
        return any(has_tensor(item) for item in obj)
    else:
        return False


def move_to_device(obj, cuda_device: int):
    """
    Given a structure (possibly) containing Tensors on the CPU,
    move all the Tensors to the specified GPU (or do nothing, if they should be on the CPU).
    """

    if cuda_device < 0 or not has_tensor(obj):
        return obj
    elif isinstance(obj, torch.Tensor):
        return obj.cuda(cuda_device)
    elif isinstance(obj, dict):
        return {key: move_to_device(value, cuda_device) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [move_to_device(item, cuda_device) for item in obj]
    elif isinstance(obj, tuple) and hasattr(obj, "_fields"):
        # This is the best way to detect a NamedTuple, it turns out.
        return obj.__class__(*(move_to_device(item, cuda_device) for item in obj))
    elif isinstance(obj, tuple):
        return tuple(move_to_device(item, cuda_device) for item in obj)
    else:
        return obj



In [8]:
## imports and helpers for training

import random
import math
import numpy as np
import torch
import copy
from pathlib import Path
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader, DistributedSampler, RandomSampler, SequentialSampler
#from tqdm import tqdm
from tqdm.auto import tqdm, trange
import sys
import os
sys.path.append("drive/MyDrive/ColabNotebooks/anli/src")

import config
from utils import common, list_dict_data_tool, save_tool
#from flint.data_utils.fields import RawFlintField, LabelFlintField, ArrayIndexFlintField

import pprint
pp = pprint.PrettyPrinter(indent=2)

MODEL_CLASSES = {
    "bert-base": {
        "model_name": "bert-base-uncased",
        "tokenizer": BertTokenizer,
        "sequence_classification": BertForSequenceClassification,
        # "padding_token_value": 0,
        "padding_segement_value": 0,
        "padding_att_value": 0,
        "do_lower_case": True,
    }
}

registered_path = {
    'snli_train': config.PRO_ROOT / "data/build/snli/train.jsonl",
    'snli_dev': config.PRO_ROOT / "data/build/snli/dev.jsonl",
    'snli_test': config.PRO_ROOT / "data/build/snli/test.jsonl",

    'mnli_train': config.PRO_ROOT / "data/build/mnli/train.jsonl",
    'mnli_m_dev': config.PRO_ROOT / "data/build/mnli/m_dev.jsonl",
    'mnli_mm_dev': config.PRO_ROOT / "data/build/mnli/mm_dev.jsonl",

    'fever_train': config.PRO_ROOT / "data/build/fever_nli/train.jsonl",
    'fever_dev': config.PRO_ROOT / "data/build/fever_nli/dev.jsonl",
    'fever_test': config.PRO_ROOT / "data/build/fever_nli/test.jsonl",

    'anli_r1_train': config.PRO_ROOT / "data/build/anli/r1/train.jsonl",
    'anli_r1_dev': config.PRO_ROOT / "data/build/anli/r1/dev.jsonl",
    'anli_r1_test': config.PRO_ROOT / "data/build/anli/r1/test.jsonl",

    'anli_r2_train': config.PRO_ROOT / "data/build/anli/r2/train.jsonl",
    'anli_r2_dev': config.PRO_ROOT / "data/build/anli/r2/dev.jsonl",
    'anli_r2_test': config.PRO_ROOT / "data/build/anli/r2/test.jsonl",

    'anli_r3_train': config.PRO_ROOT / "data/build/anli/r3/train.jsonl",
    'anli_r3_dev': config.PRO_ROOT / "data/build/anli/r3/dev.jsonl",
    'anli_r3_test': config.PRO_ROOT / "data/build/anli/r3/test.jsonl",
}

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

def sample_data_list(d_list, ratio):
    if ratio <= 0:
        raise ValueError("Invalid training weight ratio. Please change --train_weights.")
    upper_int = int(math.ceil(ratio))
    if upper_int == 1:
        return d_list # if ratio is 1 then we just return the data list
    else:
        sampled_d_list = []
        for _ in range(upper_int):
            sampled_d_list.extend(copy.deepcopy(d_list))
        if np.isclose(ratio, upper_int):
            return sampled_d_list
        else:
            sampled_length = int(ratio * len(d_list))
            random.shuffle(sampled_d_list)
            return sampled_d_list[:sampled_length]

class NLITransform(object):
    def __init__(self, model_name, tokenizer, max_length=None):
        self.model_name = model_name
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, sample):
        processed_sample = dict()
        processed_sample['uid'] = sample['uid']
        processed_sample['gold_label'] = sample['label']
        processed_sample['y'] = nli_label2index[sample['label']]

        # premise: str = sample['premise']
        premise: str = sample['context'] if 'context' in sample else sample['premise']
        hypothesis: str = sample['hypothesis']

        if premise.strip() == '':
            premise = 'empty'

        if hypothesis.strip() == '':
            hypothesis = 'empty'

        tokenized_input_seq_pair = self.tokenizer.encode_plus(premise, hypothesis,
                                                              max_length=self.max_length,
                                                              return_token_type_ids=True, truncation=True)

        processed_sample.update(tokenized_input_seq_pair)

        return processed_sample

def build_eval_dataset_loader_and_sampler(d_list, data_transformer, batching_schema, batch_size_per_gpu_eval):
    d_dataset = NLIDataset(d_list, data_transformer)
    d_sampler = SequentialSampler(d_dataset)
    d_dataloader = DataLoader(dataset=d_dataset,
                              batch_size=batch_size_per_gpu_eval,
                              shuffle=False,  #
                              num_workers=0,
                              pin_memory=True,
                              sampler=d_sampler,
                              collate_fn=BaseBatchBuilder(batching_schema))  #
    return d_dataset, d_sampler, d_dataloader

class NLIDataset(Dataset):
    def __init__(self, data_list, transform) -> None:
        super().__init__()
        self.d_list = data_list
        self.len = len(self.d_list)
        self.transform = transform

    def __getitem__(self, index: int):
        return self.transform(self.d_list[index])

    # you should write schema for each of the input elements

    def __len__(self) -> int:
        return self.len

nli_label2index = {
    'e': 0,
    'n': 1,
    'c': 2,
    'h': -1,
}

In [9]:
## build train set & set model args

# Args:
local_rank = 0 # this is the gpu rank, we only have 1
model_class_name='bert-base' # changed from roberta-base for faster training time https://towardsdatascience.com/bert-roberta-distilbert-xlnet-which-one-to-use-3d5ab82ba5f8
adam_epsilon=1e-08
cpu=False
debug_mode=False
epochs=2
eval_data='anli_r1_dev:none,anli_r2_dev:none,anli_r3_dev:none'
eval_frequency=2000
experiment_name='roberta-base|snli+mnli+fnli+r1*10+r2*20+r3*10|nli'
fp16=False
fp16_opt_level='O1'
global_iteration=0
gpus_per_node=1
gradient_accumulation_steps=4
learning_rate=1e-05
max_grad_norm=1.0
max_length=156
node_rank=0
num_nodes=1
per_gpu_eval_batch_size=16
per_gpu_train_batch_size=4
resume_path=None
sampler_seed=-1
save_prediction=True
seed=1
single_gpu=True
total_step=-1
#train_data='anli_r1_train:none,anli_r2_train:none,anli_r3_train:none'
train_data='anli_r1_train:none'
#train_weights='10,20,10'
train_weights='10'
warmup_steps=-1
weight_decay=0.0
world_size=1
args_dict = {'local_rank':local_rank,'model_class_name':model_class_name,'adam_epsilon':adam_epsilon,'cpu':cpu,'debug_mode':debug_mode,'epochs':epochs,'eval_data':eval_data,'eval_frequency':eval_frequency,'experiment_name':experiment_name,'fp16':fp16,'fp16_opt_level':fp16_opt_level,'global_iteration':global_iteration,'gpus_per_node':gpus_per_node,'gradient_accumulation_steps':gradient_accumulation_steps,'learning_rate':learning_rate,'max_grad_norm':max_grad_norm,'max_length':max_length,'node_rank':node_rank,'num_nodes':num_nodes,'per_gpu_eval_batch_size':per_gpu_eval_batch_size,'per_gpu_train_batch_size':per_gpu_train_batch_size,'resume_path':resume_path,'sampler_seed':sampler_seed,'save_prediction:':save_prediction,'seed':seed,'single_gpu':single_gpu,'total_step':total_step,'train_data':train_data,'train_weights':train_weights,'warmup_steps':warmup_steps,'weight_decay':weight_decay,'world_size': world_size }

global_rank = node_rank * gpus_per_node + local_rank
local_rank = local_rank
# warmup_steps = 20
debug_count = 1000

if total_step > 0:
  num_epoch = 10000  # if we set total step, num_epoch will be forever.
else:
  num_epoch = epochs

actual_train_batch_size = world_size * per_gpu_train_batch_size * gradient_accumulation_steps
actual_train_batch_size = actual_train_batch_size

set_seed(seed)
num_labels = 3      # we are doing NLI so we set num_labels = 3, for other task we can change this value.

max_length = max_length

model_class_item = MODEL_CLASSES[model_class_name]
model_name = model_class_item['model_name']
do_lower_case = model_class_item['do_lower_case'] if 'do_lower_case' in model_class_item else False

tokenizer = model_class_item['tokenizer'].from_pretrained(model_name,
                                                        cache_dir=str(config.PRO_ROOT / "trans_cache"),
                                                        do_lower_case=do_lower_case)

model = model_class_item['sequence_classification'].from_pretrained(model_name,
                                                                  cache_dir=str(config.PRO_ROOT / "trans_cache"),
                                                                  num_labels=num_labels)

padding_token_value = tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0]
padding_segement_value = model_class_item["padding_segement_value"]
padding_att_value = model_class_item["padding_att_value"]
left_pad = model_class_item['left_pad'] if 'left_pad' in model_class_item else False

batch_size_per_gpu_train = per_gpu_train_batch_size
batch_size_per_gpu_eval = per_gpu_eval_batch_size

if not cpu and not single_gpu:
  dist.init_process_group(
      backend='nccl',
      init_method='env://',
      world_size=world_size,
      rank=global_rank
  )

train_data_str = train_data
train_data_weights_str = train_weights
eval_data_str = eval_data

train_data_name = []
train_data_path = []
train_data_list = []
train_data_weights = []

eval_data_name = []
eval_data_path = []
eval_data_list = []

train_data_named_path = train_data_str.split(',')
weights_str = train_data_weights_str.split(',') if train_data_weights_str is not None else None

eval_data_named_path = eval_data_str.split(',')

for named_path in train_data_named_path:
  ind = named_path.find(':')
  name = named_path[:ind]
  path = name[ind + 1:]
  if name in registered_path: # breaks here
      d_list = common.load_jsonl(registered_path[name])
  else:
      d_list = common.load_jsonl(path)

  train_data_name.append(name)
  train_data_path.append(path)

  train_data_list.append(d_list)

if weights_str is not None:
  for weights in weights_str:
      train_data_weights.append(float(weights))
else:
  for i in range(len(train_data_list)):
      train_data_weights.append(1)

for named_path in eval_data_named_path:
  ind = named_path.find(':')
  name = named_path[:ind]
  path = name[ind + 1:]
  if name in registered_path:
      d_list = common.load_jsonl(registered_path[name])
  else:
      d_list = common.load_jsonl(path)
  eval_data_name.append(name)
  eval_data_path.append(path)

  eval_data_list.append(d_list)

assert len(train_data_weights) == len(train_data_list)

batching_schema = {
  'uid': RawFlintField(),
  'y': LabelFlintField(),
  'input_ids': ArrayIndexFlintField(pad_idx=padding_token_value, left_pad=left_pad),
  'token_type_ids': ArrayIndexFlintField(pad_idx=padding_segement_value, left_pad=left_pad),
  'attention_mask': ArrayIndexFlintField(pad_idx=padding_att_value, left_pad=left_pad),
}

data_transformer = NLITransform(model_name, tokenizer, max_length)
# data_transformer = NLITransform(model_name, tokenizer, max_length, with_element=True)

eval_data_loaders = []
for eval_d_list in eval_data_list:
  d_dataset, d_sampler, d_dataloader = build_eval_dataset_loader_and_sampler(eval_d_list, data_transformer,
                                                                              batching_schema,
                                                                              batch_size_per_gpu_eval)
  eval_data_loaders.append(d_dataloader)

# Estimate the training size:
training_list = []
for i in range(len(train_data_list)):
  print("Build Training Data ...")
  train_d_list = train_data_list[i]
  train_d_name = train_data_name[i]
  train_d_weight = train_data_weights[i]
  cur_train_list = sample_data_list(train_d_list, train_d_weight)  # change later  # we can apply different sample strategy here.
  print(f"Data Name:{train_d_name}; Weight: {train_d_weight}; "
        f"Original Size: {len(train_d_list)}; Sampled Size: {len(cur_train_list)}")
  training_list.extend(cur_train_list)
estimated_training_size = len(training_list)
print("Estimated training size:", estimated_training_size)
# Estimate the training size ends:

# t_total = estimated_training_size // gradient_accumulation_steps * num_epoch
# t_total = estimated_training_size * num_epoch // actual_train_batch_size
if total_step <= 0:
  t_total = estimated_training_size * num_epoch // actual_train_batch_size
else:
  t_total = total_step

if warmup_steps <= 0:  # set the warmup steps to 0.1 * total step if the given warmup step is -1.
  warmup_steps = int(t_total * 0.1)

if not cpu:
  torch.cuda.set_device(local_rank)
  model.cuda(local_rank)

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
  {
      "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
      "weight_decay": weight_decay,
  },
  {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(
  optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total
)

global_step = 0

if resume_path:
  print("Resume Training")
  global_step = global_iteration
  print("Resume Global Step: ", global_step)
  model.load_state_dict(torch.load(str(Path(resume_path) / "model.pt"), map_location=torch.device('cpu')))
  optimizer.load_state_dict(torch.load(str(Path(resume_path) / "optimizer.pt"), map_location=torch.device('cpu')))
  scheduler.load_state_dict(torch.load(str(Path(resume_path) / "scheduler.pt"), map_location=torch.device('cpu')))
  print("State Resumed")

if fp16:
  try:
      from apex import amp
  except ImportError:
      raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
  model, optimizer = amp.initialize(model, optimizer, opt_level=fp16_opt_level)

if not cpu and not single_gpu:
  model = nn.parallel.DistributedDataParallel(model, device_ids=[local_rank],
                                              output_device=local_rank, find_unused_parameters=True)

#args_dict = dict(vars(args))
file_path_prefix = '.'
if global_rank in [-1, 0]:
  print("Total Steps:", t_total)
  total_step = t_total
  print("Warmup Steps:", warmup_steps)
  print("Actual Training Batch Size:", actual_train_batch_size)
  print("Arguments", pp.pprint(args_dict))

is_finished = False

# Let build the logger and log everything before the start of the first training epoch.
if global_rank in [-1, 0]:  # only do logging if we use cpu or global_rank=0
  resume_prefix = ""
  # if resume_path:
  #     resume_prefix = "resumed_"

  if not debug_mode:
      file_path_prefix, date = save_tool.gen_file_prefix(f"{experiment_name}")
      # # # Create Log File
      # Save the source code.
      script_name = "colab_training" #os.path.basename(__file__)
      with open(os.path.join(file_path_prefix, script_name), 'w') as out_f, open("drive/MyDrive/ColabNotebooks/anli/colab_train_log.txt", 'r') as it:
          out_f.write(it.read())
          out_f.flush()

      # Save option file
      common.save_json(args_dict, os.path.join(file_path_prefix, "json"))
      checkpoints_path = Path(file_path_prefix) / "checkpoints"
      if not checkpoints_path.exists():
          checkpoints_path.mkdir()
      prediction_path = Path(file_path_prefix) / "predictions"
      if not prediction_path.exists():
          prediction_path.mkdir()

      # if this is a resumed, then we save the resumed path.
      if resume_path:
          with open(os.path.join(file_path_prefix, "resume_log.txt"), 'w') as out_f:
              out_f.write(str(resume_path))
              out_f.flush()

# print(f"Global Rank:{global_rank} ### ", 'Init!')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Load Jsonl: /content/drive/MyDrive/ColabNotebooks/anli/data/build/anli/r1/train.jsonl
Load Jsonl: /content/drive/MyDrive/ColabNotebooks/anli/data/build/anli/r1/dev.jsonl


1000it [00:00, 60200.71it/s]
1000it [00:00, 55592.72it/s]
1200it [00:00, 59737.99it/s]


Load Jsonl: /content/drive/MyDrive/ColabNotebooks/anli/data/build/anli/r2/dev.jsonl
Load Jsonl: /content/drive/MyDrive/ColabNotebooks/anli/data/build/anli/r3/dev.jsonl
Build Training Data ...
Data Name:anli_r1_train; Weight: 10.0; Original Size: 16946; Sampled Size: 169460
Estimated training size: 169460
Total Steps: 21182
Warmup Steps: 2118
Actual Training Batch Size: 16
{ 'adam_epsilon': 1e-08,
  'cpu': False,
  'debug_mode': False,
  'epochs': 2,
  'eval_data': 'anli_r1_dev:none,anli_r2_dev:none,anli_r3_dev:none',
  'eval_frequency': 2000,
  'experiment_name': 'roberta-base|snli+mnli+fnli+r1*10+r2*20+r3*10|nli',
  'fp16': False,
  'fp16_opt_level': 'O1',
  'global_iteration': 0,
  'gpus_per_node': 1,
  'gradient_accumulation_steps': 4,
  'learning_rate': 1e-05,
  'local_rank': 0,
  'max_grad_norm': 1.0,
  'max_length': 156,
  'model_class_name': 'bert-base',
  'node_rank': 0,
  'num_nodes': 1,
  'per_gpu_eval_batch_size': 16,
  'per_gpu_train_batch_size': 4,
  'resume_path': None,
 

In [ ]:
for epoch in tqdm(range(num_epoch), desc="Epoch", disable=global_rank not in [-1, 0]):
  # Let's build up training dataset for this epoch
  training_list = []
  for i in range(len(train_data_list)):
      print("Build Training Data ...")
      train_d_list = train_data_list[i]
      train_d_name = train_data_name[i]
      train_d_weight = train_data_weights[i]
      cur_train_list = sample_data_list(train_d_list, train_d_weight)  # change later  # we can apply different sample strategy here.
      print(f"Data Name:{train_d_name}; Weight: {train_d_weight}; "
            f"Original Size: {len(train_d_list)}; Sampled Size: {len(cur_train_list)}")
      training_list.extend(cur_train_list)

  random.shuffle(training_list)
  train_dataset = NLIDataset(training_list, data_transformer)

  train_sampler = SequentialSampler(train_dataset)
  if not cpu and not single_gpu:
      print("Use distributed sampler.")
      train_sampler = DistributedSampler(train_dataset, world_size, global_rank,
                                          shuffle=True)

  train_dataloader = DataLoader(dataset=train_dataset,
                                batch_size=batch_size_per_gpu_train,
                                shuffle=False,  #
                                num_workers=0,
                                pin_memory=True,
                                sampler=train_sampler,
                                collate_fn=BaseBatchBuilder(batching_schema))  #

  if not cpu and not single_gpu:
      if sampler_seed == -1:
          train_sampler.set_epoch(epoch)  # setup the epoch to ensure random sampling at each epoch
      else:
          train_sampler.set_epoch(epoch + sampler_seed)

  for forward_step, batch in enumerate(tqdm(train_dataloader, desc="Iteration",
                                            disable=global_rank not in [-1, 0]), 0):
      model.train()

      batch = move_to_device(batch, local_rank)
      # print(batch['input_ids'], batch['y'])
      if model_class_name in ["distilbert", "bart-large"]:
          outputs = model(batch['input_ids'],
                          attention_mask=batch['attention_mask'],
                          labels=batch['y'])
      else:
          outputs = model(batch['input_ids'],
                          attention_mask=batch['attention_mask'],
                          token_type_ids=batch['token_type_ids'],
                          labels=batch['y'])
      loss, logits = outputs[:2]
      # print(debug_node_info(args), loss, logits, batch['uid'])
      # print(debug_node_info(args), loss, batch['uid'])

      # Accumulated loss
      if gradient_accumulation_steps > 1:
          loss = loss / gradient_accumulation_steps

      # if this forward step need model updates
      # handle fp16
      if fp16:
          with amp.scale_loss(loss, optimizer) as scaled_loss:
              scaled_loss.backward()
      else:
          loss.backward()

          # Gradient clip: if max_grad_norm < 0
      if (forward_step + 1) % gradient_accumulation_steps == 0:
          if max_grad_norm > 0:
              if fp16:
                  torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), max_grad_norm)
              else:
                  torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

          optimizer.step()
          scheduler.step()  # Update learning rate schedule
          model.zero_grad()

          global_step += 1

          if global_rank in [-1, 0] and eval_frequency > 0 and global_step % eval_frequency == 0:
              r_dict = dict()
              # Eval loop:
              for i in range(len(eval_data_name)):
                  cur_eval_data_name = eval_data_name[i]
                  cur_eval_data_list = eval_data_list[i]
                  cur_eval_dataloader = eval_data_loaders[i]
                  # cur_eval_raw_data_list = eval_raw_data_list[i]

                  evaluation_dataset(args, cur_eval_dataloader, cur_eval_data_list, model, r_dict,
                                      eval_name=cur_eval_data_name)

              # saving checkpoints
              current_checkpoint_filename = \
                  f'e({epoch})|i({global_step})'

              for i in range(len(eval_data_name)):
                  cur_eval_data_name = eval_data_name[i]
                  current_checkpoint_filename += \
                      f'|{cur_eval_data_name}#({round(r_dict[cur_eval_data_name]["acc"], 4)})'

              if not debug_mode:
                  # save model:
                  model_output_dir = checkpoints_path / current_checkpoint_filename
                  if not model_output_dir.exists():
                      model_output_dir.mkdir()
                  model_to_save = (
                      model.module if hasattr(model, "module") else model
                  )  # Take care of distributed/parallel training

                  torch.save(model_to_save.state_dict(), str(model_output_dir / "model.pt"))
                  torch.save(optimizer.state_dict(), str(model_output_dir / "optimizer.pt"))
                  torch.save(scheduler.state_dict(), str(model_output_dir / "scheduler.pt"))

              # save prediction:
              if not debug_mode and save_prediction:
                  cur_results_path = prediction_path / current_checkpoint_filename
                  if not cur_results_path.exists():
                      cur_results_path.mkdir(parents=True)
                  for key, item in r_dict.items():
                      common.save_jsonl(item['predictions'], cur_results_path / f"{key}.jsonl")

                  # avoid saving too many things
                  for key, item in r_dict.items():
                      del r_dict[key]['predictions']
                  common.save_json(r_dict, cur_results_path / "results_dict.json", indent=2)

          if total_step > 0 and global_step == t_total:
              # if we set total step and global step s t_total.
              is_finished = True
              break

  # End of epoch evaluation.
  if global_rank in [-1, 0] and total_step <= 0:
      r_dict = dict()
      # Eval loop:
      for i in range(len(eval_data_name)):
          cur_eval_data_name = eval_data_name[i]
          cur_eval_data_list = eval_data_list[i]
          cur_eval_dataloader = eval_data_loaders[i]
          # cur_eval_raw_data_list = eval_raw_data_list[i]

          evaluation_dataset(args, cur_eval_dataloader, cur_eval_data_list, model, r_dict,
                              eval_name=cur_eval_data_name)

      # saving checkpoints
      current_checkpoint_filename = \
          f'e({epoch})|i({global_step})'

      for i in range(len(eval_data_name)):
          cur_eval_data_name = eval_data_name[i]
          current_checkpoint_filename += \
              f'|{cur_eval_data_name}#({round(r_dict[cur_eval_data_name]["acc"], 4)})'

      if not debug_mode:
          # save model:
          model_output_dir = checkpoints_path / current_checkpoint_filename
          if not model_output_dir.exists():
              model_output_dir.mkdir()
          model_to_save = (
              model.module if hasattr(model, "module") else model
          )  # Take care of distributed/parallel training

          torch.save(model_to_save.state_dict(), str(model_output_dir / "model.pt"))
          torch.save(optimizer.state_dict(), str(model_output_dir / "optimizer.pt"))
          torch.save(scheduler.state_dict(), str(model_output_dir / "scheduler.pt"))

      # save prediction:
      if not debug_mode and save_prediction:
          cur_results_path = prediction_path / current_checkpoint_filename
          if not cur_results_path.exists():
              cur_results_path.mkdir(parents=True)
          for key, item in r_dict.items():
              common.save_jsonl(item['predictions'], cur_results_path / f"{key}.jsonl")

          # avoid saving too many things
          for key, item in r_dict.items():
              del r_dict[key]['predictions']
          common.save_json(r_dict, cur_results_path / "results_dict.json", indent=2)

  if is_finished:
      break


Build Training Data ...
Data Name:anli_r1_train; Weight: 10.0; Original Size: 16946; Sampled Size: 169460
